#Set the environment

In [1]:
#mount google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#Install requirements.txt.
#Given path might not work for you as your location might be different
#Find the requirements.txt and copy path and replace this path

!pip install -r /content/drive/MyDrive/IndoNLP-main/IndoNLP-main/FakeNews-master/requirements.txt

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.8/252.8 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.4/276.4 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.4/536.4 kB 23.2 MB/s eta 0:00:00
  Created wheel for ipaddr: filename=ipaddr-2.2.0-py3-none-any.whl size=18281 sha256=90d21c8851d26e8f477d483eb8853bcfe6256e2179131e75fb985d4e37a959e2
  Stored in directory: /root/.cache/pip/wheels/95/99/bd/70fe0c01fa82e66ac6e05659f7d1c8bd61770d582c63c36824
  Created wheel for slugify: filename=slugify-0.0.1-py3-none-any.whl size=1882 sha256=247ee18bb418ae3dda4eb35eb5c4f838636f8c877e4720e0a45405

In [3]:
#These are all the imports necessary. If you encounter anything missing just import here (hopefully that won't be the case).

import os
import pandas as pd
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import string
import pickle
import torch
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import sys

#sys.append Basic folder and Helper folder. Both can be found in the Models subfolder. copy your path and replace it.

sys.path.append('/content/drive/MyDrive/IndoNLP-main/IndoNLP-main/FakeNews-master/Models/Basic')
sys.path.append('/content/drive/MyDrive/IndoNLP-main/IndoNLP-main/FakeNews-master/Models/Helper')

import load_feature
import helper

In [4]:
#Change our directory where n-gram.py is located.
#Given path might not work for you as your location might be different
#Find the "Basic" sub-folder in the "Models" folder where n-gram.py is located and replace this path

%cd drive/MyDrive/IndoNLP-main/IndoNLP-main/FakeNews-master/Models/Basic

/content/drive/MyDrive/IndoNLP-main/IndoNLP-main/FakeNews-master/Models/Basic


#Train model

In [5]:
#Run our model
#Change exp_name and model_name according to your work
#For example : " !python n-gram.py Unigram SVM -s ""
#Refer to https://github.com/Rowan1697/FakeNews for all the available experiment names and models

!python n-gram.py Unigram SVM -s

#After your model has been trained you can find the saved RESULT in the results folder
#After your model has been trained you can find the saved MODEL in the saved-model folder
#Your saved model will have the naming convenction of "model name" + '_' + "experiment name" + ".pkl"
#For example "LR_C3-gram.pkl"

/usr/local/lib/python3.12/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['পাওয়া', 'পেয়ে', 'বিষয়টি', 'হইয়া', 'হয়েছিল'] not in stop_words.
  warnings.warn(
                                Weighted Avg         #            Non-Violence           #          Passive Violence         #          Active Violence          
                   precision    recall      f1-score  #  precision    recall      f1-score  #  precision    recall      f1-score  #  precision    recall      f1-score
Unigram            0.72         0.72        0.72      #  0.73         0.80         0.76      #  0.71         0.67         0.69      #  0.73         0.66         0.69
                                Macro 

#Testing

In [11]:
#set the path of our model and datasets
#You can find the models in saved models folder under Basic
#You can find the datasets in dataset folder under 4-2

model_path = '/content/drive/MyDrive/IndoNLP-main/IndoNLP-main/FakeNews-master/Models/Basic/Saved Models/SVM_Unigram.pkl'
train_data_path = '/content/drive/MyDrive/IndoNLP-main/IndoNLP-main/Datasets/train.csv'
test_data_path = '/content/drive/MyDrive/IndoNLP-main/IndoNLP-main/Datasets/test.csv'

In [12]:
#Prepare the test dataset

df_train = pd.read_csv(train_data_path)
df_test = pd.read_csv(test_data_path)

# Rename columns to match what load_feature.py expects
# Your columns: 'Text' → rename to 'Content'
#               'Labels' → rename to 'Label' (singular, as used in word_emb)
df_train = df_train.rename(columns={'text': 'Content', 'label': 'Label'})
df_test = df_test.rename(columns={'text': 'Content', 'label': 'Label'})

# Now create X_train and X_test as DataFrames with 'Content' column
X_train = df_train[['Content']]
X_test = df_test[['Content']]

y_test = df_test['Label'].values.ravel()

#For Unigram set a = 1, b = 1
#For Bigram set a = 2, b = 2
#For Trigram set a = 3, b = 3
#For U+B+T set a = 1, b = 3

#For C3-gram set a = 3, b = 3
#For C4-gram set a = 4, b = 4
#For C5-gram set a = 5, b = 5
#For C3+C4+C5 set a = 3, b = 5

a = 1
b = 1

#If you are using U/B/T : use wordF
#If you are using C3/C4/C5 : use charF
#For example when using C3, I have only used charF, and commented out wordF.

X_test = load_feature.tfidf_wordF(X_train, X_test, a, b)
# X_test = load_feature.tfidf_charF(X_train, X_test, a, b)

/usr/local/lib/python3.12/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['পাওয়া', 'পেয়ে', 'বিষয়টি', 'হইয়া', 'হয়েছিল'] not in stop_words.
  warnings.warn(


In [13]:
#load and Test the model

clf = pickle.load(open(model_path, 'rb'))
y_pred = clf.predict(X_test)

In [ ]:
# #get and print the results
# t, f, o, om = helper.getResult(y_test, y_pred)

# print("                                Overall               #               Fake                ")
# print("                   precision    recall      f1-score  #  precision    recall      f1-score")

# #Set the 'model' and 'experiment' names
# model = 'SVM'
# experiment = 'Unigram'
# exp = model+'_'+experiment

# res = helper.printResult(exp,o,f)
# print(res)

# path = model+"_final_results.txt"
# helper.saveResults(path, res)

# print("                                Overall               #               Fake                ")
# print("                   precision    recall      f1-score  #  precision    recall      f1-score")

# res = helper.printResult(exp,om,f)
# print(res)

# path = model+"_final_results_micro.txt"
# helper.saveResults(path, res)

In [14]:
#load and Test the model

clf = pickle.load(open(model_path, 'rb'))
y_pred = clf.predict(X_test)

class0, class1, class2, o_weighted, o_macro = helper.getResult(y_test, y_pred)

print("                                Weighted Avg         #            Non-Violence           #          Passive Violence         #          Active Violence          ")
print("                   precision    recall      f1-score  #  precision    recall      f1-score  #  precision    recall      f1-score  #  precision    recall      f1-score")

#Set the 'model' and 'experiment' names
model = 'SVM'
experiment = 'Unigram'
exp = model+'_'+experiment

res = helper.printResult(exp, o_weighted, class0, class1, class2)
print(res)

path = model+"_final_results.txt"
helper.saveResults(path, res)

print("                                Macro Avg            #            Non-Violence           #          Passive Violence         #          Active Violence          ")
print("                   precision    recall      f1-score  #  precision    recall      f1-score  #  precision    recall      f1-score  #  precision    recall      f1-score")

res = helper.printResult(exp, o_macro, class0, class1, class2)
print(res)

path = model+"_final_results_macro.txt"
helper.saveResults(path, res)

                                Weighted Avg         #            Non-Violence           #          Passive Violence         #          Active Violence          
                   precision    recall      f1-score  #  precision    recall      f1-score  #  precision    recall      f1-score  #  precision    recall      f1-score
SVM_Unigram        0.73         0.73        0.73      #  0.75         0.80         0.77      #  0.71         0.68         0.69      #  0.73         0.67         0.70
                                Macro Avg            #            Non-Violence           #          Passive Violence         #          Active Violence          
                   precision    recall      f1-score  #  precision    recall      f1-score  #  precision    recall      f1-score  #  precision    recall      f1-score
SVM_Unigram        0.73         0.72        0.72      #  0.75         0.80         0.77      #  0.71         0.68         0.69      #  0.73         0.67         0.70
